### Import

In [1]:
!pip install lightgbm

   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 1.5/1.5 MB 41.8 MB/s eta 0:00:00


In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import OrdinalEncoder

import optuna
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

### Data Load

In [2]:
train = pd.read_csv('./Data/train.csv').drop(columns=['ID'])
test = pd.read_csv('./Data/test.csv').drop(columns=['ID'])

In [3]:
X = train.drop('임신 성공 여부', axis=1)
y = train['임신 성공 여부']

### Data Pre-processing

In [4]:
categorical_columns = [
    "시술 시기 코드",
    "시술 당시 나이",
    "시술 유형",
    "특정 시술 유형",
    "배란 자극 여부",
    "배란 유도 유형",
    "단일 배아 이식 여부",
    "착상 전 유전 검사 사용 여부",
    "착상 전 유전 진단 사용 여부",
    "남성 주 불임 원인",
    "남성 부 불임 원인",
    "여성 주 불임 원인",
    "여성 부 불임 원인",
    "부부 주 불임 원인",
    "부부 부 불임 원인",
    "불명확 불임 원인",
    "불임 원인 - 난관 질환",
    "불임 원인 - 남성 요인",
    "불임 원인 - 배란 장애",
    "불임 원인 - 여성 요인",
    "불임 원인 - 자궁경부 문제",
    "불임 원인 - 자궁내막증",
    "불임 원인 - 정자 농도",
    "불임 원인 - 정자 면역학적 요인",
    "불임 원인 - 정자 운동성",
    "불임 원인 - 정자 형태",
    "배아 생성 주요 이유",
    "총 시술 횟수",
    "클리닉 내 총 시술 횟수",
    "IVF 시술 횟수",
    "DI 시술 횟수",
    "총 임신 횟수",
    "IVF 임신 횟수",
    "DI 임신 횟수",
    "총 출산 횟수",
    "IVF 출산 횟수",
    "DI 출산 횟수",
    "난자 출처",
    "정자 출처",
    "난자 기증자 나이",
    "정자 기증자 나이",
    "동결 배아 사용 여부",
    "신선 배아 사용 여부",
    "기증 배아 사용 여부",
    "대리모 여부",
    "PGD 시술 여부",
    "PGS 시술 여부"
]

In [5]:
# 카테고리형 컬럼들을 문자열로 변환
for col in categorical_columns:
    X[col] = X[col].astype(str)
    test[col] = test[col].astype(str)

In [6]:
ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

X_train_encoded = X.copy()
X_train_encoded[categorical_columns] = ordinal_encoder.fit_transform(X[categorical_columns])

X_test_encoded = test.copy()
X_test_encoded[categorical_columns] = ordinal_encoder.transform(test[categorical_columns])

In [7]:
numeric_columns = [
    "임신 시도 또는 마지막 임신 경과 연수",
    "총 생성 배아 수",
    "미세주입된 난자 수",
    "미세주입에서 생성된 배아 수",
    "이식된 배아 수",
    "미세주입 배아 이식 수",
    "저장된 배아 수",
    "미세주입 후 저장된 배아 수",
    "해동된 배아 수",
    "해동 난자 수",
    "수집된 신선 난자 수",
    "저장된 신선 난자 수",
    "혼합된 난자 수",
    "파트너 정자와 혼합된 난자 수",
    "기증자 정자와 혼합된 난자 수",
    "난자 채취 경과일",
    "난자 해동 경과일",
    "난자 혼합 경과일",
    "배아 이식 경과일",
    "배아 해동 경과일"
]

In [8]:
numeric_columns = [col for col in numeric_columns if col in X.columns]

X_train_encoded[numeric_columns] = X_train_encoded[numeric_columns].fillna(0)
X_test_encoded[numeric_columns] = X_test_encoded[numeric_columns].fillna(0)

In [9]:
from collections import Counter

# 클래스별 샘플 개수 출력
class_counts = Counter(y)
print(class_counts)


Counter({0: 190123, 1: 66228})


In [10]:
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_encoded, y, test_size=0.2, random_state=42, stratify=y
)

### Train

Lightgbm

In [33]:
from lightgbm import early_stopping, log_evaluation

# 피처 이름 공백 제거
X.columns = X.columns.str.replace(" ", "_")

def objective(trial):
    # '시술 당시 나이' 가중치를 Optuna에서 탐색하도록 설정
    age_weight = trial.suggest_float("age_weight", 1.0, 3.0, step=0.1)

    params = {
        # 'objective': 'binary',
        # 'metric': 'auc',
        # 'boosting_type': 'gbdt',
        # 'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.1),
        # 'num_leaves': trial.suggest_int('num_leaves', 20, 60),
        # 'max_depth': trial.suggest_int('max_depth', 5, 15),
        # 'min_child_samples': trial.suggest_int('min_child_samples', 10, 40),
        # 'subsample': trial.suggest_uniform('subsample', 0.6, 1.0),
        # 'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.6, 1.0),
        # 'feature_fraction': trial.suggest_uniform('feature_fraction', 0.6, 1.0),
        # 'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.6, 1.0),
        # 'verbosity': -1
        'objective': 'binary',
        'metric': 'auc',
        'boosting_type': 'gbdt',
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
        'num_leaves': trial.suggest_int('num_leaves', 10, 100),
        'max_depth': trial.suggest_int('max_depth', 3, 20),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 50),
        'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1.0),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.5, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.5, 1.0),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-9, 100.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-9, 100.0),
        'min_split_gain': trial.suggest_loguniform('min_split_gain', 1e-9, 1.0),
        'feature_weights': [age_weight if col == '시술 당시 나이' else 1 for col in X_train.columns],
        'random_seed': 42,        
        'verbosity': -1
    }
    
    model = lgb.LGBMClassifier(**params, n_estimators=2000)
    model.fit(
        X_train, y_train,
        eval_set=[(X_valid, y_valid)],
        eval_metric='auc',
        callbacks=[
            early_stopping(100),  # 100번 연속 성능 개선 없으면 종료
            # log_evaluation(10)   # 10번마다 로그 출력
        ]
    )
    
    preds = model.predict_proba(X_valid)[:, 1]
    auc = roc_auc_score(y_valid, preds)
    return auc


In [34]:
import warnings

# 불필요한 경고 제거
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
warnings.filterwarnings("ignore", category=UserWarning, module="lightbgm")

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10, show_progress_bar=True)

[I 2025-02-22 23:41:35,176] A new study created in memory with name: no-name-39f76a78-3b9b-4af4-8900-aa1027a8260f


  0%|          | 0/10 [00:00<?, ?it/s]

Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[98]	valid_0's auc: 0.73731
[I 2025-02-22 23:42:37,619] Trial 0 finished with value: 0.7373095640349616 and parameters: {'age_weight': 2.9000000000000004, 'learning_rate': 0.04768019133898593, 'num_leaves': 69, 'max_depth': 17, 'min_child_samples': 34, 'subsample': 0.9775721931730422, 'colsample_bytree': 0.9029820171987928, 'feature_fraction': 0.9308139998907115, 'bagging_fraction': 0.9202077234763084, 'reg_alpha': 0.00021945772454386487, 'reg_lambda': 4.6136101075294036e-05, 'min_split_gain': 0.19805359904068562}. Best is trial 0 with value: 0.7373095640349616.
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[29]	valid_0's auc: 0.736805
[I 2025-02-22 23:43:24,018] Trial 1 finished with value: 0.7368045143421164 and parameters: {'age_weight': 1.8, 'learning_rate': 0.22572718484837898, 'num_leaves': 53, 'max_depth': 10, 'min_child_samples': 40

In [69]:
import joblib
# 저장된 모델 불러오기
final_model = joblib.load('./LightGBM_boosting2.pkl')

# 추가 학습을 위해 warm_start 활성화
final_model.warm_start = True
final_model.n_estimators += 50  # 추가 부스팅 라운드 개수 설정

# 추가 학습 진행 (적절한 X_train, y_train 데이터를 사용)
final_model.fit(X_train_encoded, y)

LGBMClassifier(colsample_bytree=0.6283518615400887,
               learning_rate=0.020414102740183366, max_depth=9,
               min_child_samples=14, n_estimators=1070, num_leaves=35,
               reg_alpha=9.422816220857865, reg_lambda=3.965116334727029e-05,
               subsample=0.8361505791254964)

In [28]:
# 최적 하이퍼파라미터로 최종 모델 학습
best_params = study.best_trial.params
final_model = lgb.LGBMClassifier(**best_params)

final_model.fit(X_train_encoded, y)

LGBMClassifier(age_weight=2.0, bagging_fraction=0.6921787187699319,
               colsample_bytree=0.994244025070502,
               feature_fraction=0.5869672193341657,
               learning_rate=0.011136143607469865, max_depth=14,
               min_child_samples=48, min_split_gain=7.116704646598603e-09,
               num_leaves=46, reg_alpha=1.8221154640064686e-08,
               reg_lambda=41.46717792373384, subsample=0.6910015982795528)

In [70]:
from sklearn.metrics import accuracy_score, roc_auc_score

# 예측
y_train_pred = final_model.predict(X_train_encoded)
y_train_proba = final_model.predict_proba(X_train_encoded)[:, 1]  # ROC-AUC Score 계산용

# 평가
accuracy = accuracy_score(y, y_train_pred)
roc_auc = roc_auc_score(y, y_train_proba)

# 출력
print(f"Accuracy: {accuracy:.4f}")
print(f"ROC-AUC Score: {roc_auc:.4f}")

Accuracy: 0.7520
ROC-AUC Score: 0.7534


In [71]:
import joblib

# Optuna 최적화 실행 후 최적의 study 저장
joblib.dump(final_model, "LightGBM_boosting5.pkl")

['LightGBM_boosting5.pkl']

### Predict

In [72]:
pred_proba = final_model.predict_proba(X_test_encoded)[:, 1]

### Submission

In [73]:
sample_submission = pd.read_csv('./Data/sample_submission.csv')
sample_submission['probability'] = pred_proba

In [74]:
sample_submission.to_csv('./LightGBM_boosting_submit5.csv', index=False)